# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install -q crewai==0.108.0 crewai_tools==0.38.1 langchain_community==0.3.20 langchain_ollama  ollama==0.4.7 python-dotenv
```

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [3]:
from crewai import Agent, Task, Crew, LLM

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [4]:
import os
from utils import get_openai_api_key, pretty_print_result
from dotenv import load_dotenv

load_dotenv('creds.env')  # Adjust path if needed


ollama_llm = LLM(
  	model="ollama/phi4:latest",
  	api_base="http://localhost:11434"
  )


## Creating Agents

In [5]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    llm=ollama_llm,
    verbose=True
)

In [6]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    llm=ollama_llm,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [7]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/Users/skasmani/Downloads/personal/github/C

In [8]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [10]:
# from crewai_tools import BaseTool
from crewai.tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [11]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [12]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [13]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [14]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [16]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=True,
    embedder={
        "provider": "ollama",
        "config": {
            "model": "mxbai-embed-large"
        }
    },
    memory=True)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: fb7dc3da-0a55-4f04-8c23-925a6bee96ab                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: ade1fc64-173e-4753-a8e4-8574ececd8f0
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: ade1fc64-173e-4753-a8e4-8574ececd8f0
       Status: Executing Task...
    └── 🤖 Agent: Sales Representative
            Status: In Progress

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


🚀 Crew: crew
└── 📋 Task: ade1fc64-173e-4753-a8e4-8574ececd8f0
       Status: Executing Task...
    └── 🤖 Agent: Sales Representative
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: ade1fc64-173e-4753-a8e4-8574ececd8f0
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (1)
└── 📋 Task: 0a42493f-72a6-4e70-a20e-30055b8b50c7
       Status: Executing Task...
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed
        └── 🔧 Used Search the internet with Serper (2)

🚀 Crew: crew
├── 📋 Task: ade1fc64-173e-4753-a8e4-8574ececd8f0
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (1)
└── 📋 Task: 0a42493f-72a6-4e70-a20e-30055b8b50c7
       Assigned to: Lead Sales Representative
       Status: ✅ Completed
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed
        └── 🔧 Used Search the internet with Serper (2)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0a42493f-72a6-4e70-a20e-30055b8b50c7                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: fb7dc3da-0a55-4f04-8c23-925a6bee96ab                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



> Entering new CrewAgentExecutor chain...
Crafting a personalized outreach campaign for Andrew Ng, the CEO of DeepLearningAI, requires a deep understanding of their company culture, recent achievements, and future goals. I need to ensure that the communication resonates with DeepLearningAI's values and showcases how our solutions can support their growth effectively.

Action: Search the internet
Action Input: {"search_query": "Recent achievements of DeepLearningAI and Andrew Ng"} 


Search results: Title: Andrew Ng: From Prodigy to Pioneer in Artificial Intelligence
Link: https://www.perplexity.ai/page/Andrew-Ng-From-sbkAB53jTc6gOL5SsYVq5A
Snippet: One of the most notable achievements during his time at Google Brain was the development of a neural network that taught itself to recognize ...
---
Title: DeepLearning.AI: Start or Advance Your Career in AI
Link: https://www.deeplearning.ai/
Snippet: Learn the skills to start or advance your AI career | World-class education | Hands-on tr

- Display the final result as Markdown.

In [19]:
from IPython.display import Markdown
# Extract the Markdown string from the CrewOutput object
markdown_content = result.raw

# Display the Markdown content
display(Markdown(markdown_content))
# Markdown(result)

**Research Insights:**

1. **Current Focus Areas of DeepLearning.AI:** 
   - Recent developments have shown a strong emphasis on generative AI and large language models (LLMs), particularly in creating applications that leverage these technologies for practical, real-world solutions.
   - There is also an ongoing interest in ethical AI practices, ensuring that the deployment of AI aligns with responsible use guidelines.

2. **Andrew Ng's Recent Initiatives:**
   - Andrew Ng has been vocal about the importance of integrating AI into business processes to drive innovation and efficiency.
   - He frequently highlights the significance of AI ethics and education, stressing that organizations must focus on both technology development and societal impacts.

**Customized Outreach Strategy:**

1. **Emphasize Generative AI Capabilities:**
   - Highlight how our generative AI solutions can integrate seamlessly with existing frameworks to enhance creativity and productivity within DeepLearning.AI’s projects.
   - Provide case studies demonstrating successful implementations in similar domains, showcasing the tangible benefits of adopting our technologies.

2. **Align with Ethical AI Practices:**
   - Address ethical considerations directly by outlining measures we've taken to ensure that our generative AI solutions comply with industry standards for responsible AI usage.
   - Offer to collaborate on developing educational content or workshops that emphasize best practices in AI ethics, potentially partnering with Andrew Ng’s initiatives.

3. **Focus on Business Integration:**
   - Demonstrate how our solutions can streamline operations and open new avenues for innovation, drawing parallels to Andrew Ng's advocacy for business-oriented AI applications.
   - Propose a pilot project tailored to DeepLearning.AI’s current objectives, providing a risk-free way to explore the potential of our offerings.

4. **Leverage Personalized Communication:**
   - Craft personalized messages that reflect Andrew Ng's recent talks and publications, ensuring our outreach resonates with his known interests and priorities.
   - Consider reaching out through platforms where he is actively engaging, such as AI conferences or webinars, to foster a direct connection.

5. **Offer Educational Partnerships:**
   - Suggest partnerships in creating educational content that DeepLearning.AI could use for its courses, potentially enhancing their curriculum with cutting-edge AI technologies.
   - Highlight opportunities for joint research initiatives that align with the mission of both our organizations.

By focusing on these tailored strategies, we can present a compelling case to Andrew Ng and DeepLearning.AI, illustrating how our generative AI solutions not only meet but exceed their current needs and objectives.